# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from IPython.display import Markdown
import os
from time import time

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)

from api_key_store import ApiKeyStore
from helper import convert_to_filename

# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
s = ApiKeyStore()

In [3]:
os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'   # gpt-4 was default, throws rate-limit error 

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory="""
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    """.replace('"',''),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory="""
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    """.replace('"',''),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [7]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [8]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [9]:
lead_profiling_task = Task(
    description="""
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    """.replace('"',''),
    expected_output="""
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    """.replace('"',''),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [10]:
personalized_outreach_task = Task(
    description="""
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    """.replace('"',''),
    expected_output="""
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    """.replace('"',''),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [11]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

ts_start = time()
result = crew.kickoff(inputs=inputs)
ts_stop = time()

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: 
        Conduct an in-depth analysis of DeepLearningAI, 
        a company in the Online Learning Platform sector 
        that recently showed interest in our solutions. 
        Utilize all available data sources 
        to compile a detailed profile, 
        focusing on key decision-makers, recent business 
        developments, and potential needs 
        that align with our offerings. 
        This task is crucial for tailoring 
        our engagement strategy effectively.

        Don't make assumptions and 
        only use information you absolutely sure about.
    


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company overview"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Fund

- Display the final result as Markdown.

In [13]:
Markdown(result)

Unfortunately, I was unable to find specific details about DeepLearningAI's recent milestones or achievements to tailor my outreach campaign effectively.

In [14]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")



Elapsed time: 24.78 sec
